In [106]:
import os
os.environ['PYTHONHASHSEED'] = '0'
import nibabel as nib
import numpy as np
import yaml

from hashlib import md5

from nibabel.streamlines import ArraySequence, Tractogram
from nibabel.streamlines.trk import TrkFile

In [122]:
def trk2seeds(config):
    np.random.seed(42)
    
    subject_dir = os.path.join("subjects", config["subject"])
    
    trk_path = os.path.join(subject_dir, config["trk_path"])
    
    hasher = md5()
    for v in config.values():
        hasher.update(str(v).encode())
    
    save_dir = os.path.join(subject_dir, "seeds", hasher.hexdigest())
    
    if os.path.exists(save_dir):
        print("Seeds with this config have been created already:\n{}".format(save_dir))
        return
    
    trk_file = nib.streamlines.load(trk_path)
    
    seeds = []
    for fiber in trk_file.tractogram.streamlines:
        seeds.append([fiber[0]])
        seeds.append([fiber[-1]])
    seeds = np.array(seeds)
    
    if config["rand_samples"] > 0:
        rand_shifts = np.random.uniform(-config["rand_radius"], config["rand_radius"],
                                        size=(len(seeds), config["rand_samples"], 3)
        )
        rand_seeds = rand_shifts + seeds.reshape(-1, 1, 3)
        rand_seeds = rand_seeds.reshape(-1, 3)
        seeds = np.vstack([seeds, rand_seeds])
      
    tractogram = Tractogram(ArraySequence(seeds), affine_to_rasmm=np.eye(4))
    
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, "seeds.trk")
    print("Saving {}".format(save_path))
    TrkFile(tractogram, trk_file.header).save(save_path)
    
    config["n_seeds"] = len(seeds)
    config_path = os.path.join(save_dir, "config" + ".yml")
    print("Saving {}".format(config_path))
    with open(config_path, "w") as file:
            yaml.dump(config, file, default_flow_style=False)
    
    return seeds

In [123]:
config=dict(
    subject="992774",
    trk_path="tracts/CA.trk",
    rand_radius=1.0, # in mm
    rand_samples=0,
)

In [125]:
seeds = trk2seeds(config)

Saving subjects/992774/seeds/4c74deb7a7d812284c1edb8197b24a35/seeds.trk
Saving subjects/992774/seeds/4c74deb7a7d812284c1edb8197b24a35/config.yml
